In [71]:
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from scipy.spatial import ConvexHull

In [72]:
#read dataset as df
df = pd.read_csv('./MT_Data/iteration results/edgedata-14.csv')
df

,new_z,new_omega
0,0.010000,0.960000
1,0.052551,0.929501
2,0.063110,0.922508
3,0.077465,0.913160
4,0.091964,0.904039
5,0.101795,0.898324
6,0.120294,0.892274
7,0.173632,0.747272
8,0.260629,0.717008
9,0.285338,0.832693


In [73]:
#get valid data from the dataset
zeta = np.array(df.new_z)
omega_ratio = np.array(df.new_omega)

edge_df = df.groupby('new_z').min('new_omega')
#edge_Df
z_coord = np.array(edge_df.index)
#z_cood
omega_coord = np.array(edge_df.new_omega)
#edge_df

In [74]:
#find the nonconvex data points
def is_point_on_right_side(A, B, C):
    x1, y1 = A
    x2, y2 = B
    x3, y3 = C

    # Calculate the slope of line AC
    slope_AC = (y3 - y1) / (x3 - x1) if x3 != x1 else float('inf')
    slope_AB = (y2 - y1) / (x2 - x1) if x2 != x1 else float('inf')
    
    # Determine the position of B relative to line AC
    if x2 > x1 and slope_AC < slope_AB:  # B is on the right side of AC if AC has positive slope
        return True
    else:
        return False

index_check = len(z_coord) - 2
Unqualified_nodes_zetall = []
Unqualified_nodes_zetal = []
Unqualified_nodes_zeta = []
Unqualified_nodes_zetar = []

Unqualified_nodes_omegall = []
Unqualified_nodes_omegal = []
Unqualified_nodes_omega = []
Unqualified_nodes_omegar = []

for i in range(index_check):
    #print (i)
    A = (z_coord[i],omega_coord[i])
    B = (z_coord[i+1],omega_coord[i+1])
    C = (z_coord[i+2],omega_coord[i+2])
    flag = is_point_on_right_side(A,B,C)
    #print(flag)
    if flag == True:
        Unqualified_nodes_zetall = np.append(Unqualified_nodes_zetall,z_coord[i-1])
        Unqualified_nodes_zetal = np.append(Unqualified_nodes_zetal,z_coord[i])
        Unqualified_nodes_zeta = np.append(Unqualified_nodes_zeta,z_coord[i+1])
        Unqualified_nodes_zetar = np.append(Unqualified_nodes_zetar,z_coord[i+2])
        
        Unqualified_nodes_omegall = np.append(Unqualified_nodes_omegall,omega_coord[i-1])
        Unqualified_nodes_omegal = np.append(Unqualified_nodes_omegal,omega_coord[i])
        Unqualified_nodes_omega = np.append(Unqualified_nodes_omega,omega_coord[i+1])
        Unqualified_nodes_omegar = np.append(Unqualified_nodes_omegar,omega_coord[i+2])
        
df_adjust_nodes = pd.DataFrame({'Unqualified_nodes_zetall':Unqualified_nodes_zetall,'Unqualified_nodes_zetaleft':Unqualified_nodes_zetal,'Unqualified_nodes_zeta':Unqualified_nodes_zeta,'Unqualified_nodes_zetaright':Unqualified_nodes_zetar,'Unqualified_nodes_omegall':Unqualified_nodes_omegall,'Unqualified_nodes_omegaleft':Unqualified_nodes_omegal,'Unqualified_nodes_omega':Unqualified_nodes_omega,'Unqualified_nodes_omegaright':Unqualified_nodes_omegar})
df_adjust_nodes
df_adjust_nodes_save = pd.DataFrame({'Unqualified_nodes_zeta':Unqualified_nodes_zeta,'Unqualified_nodes_omega':Unqualified_nodes_omega})
df_adjust_nodes_save.to_csv("tzzs_data.csv")


In [75]:
#find the adjusted points
def mid_point(A,C):
    x1, y1 = A
    x3, y3 = C
    
    mid_point = (x1 + x3)/2, (y1 + y3)/2
    print(mid_point)
    return mid_point

def gradient(M,N):
    x1, y1 = M
    x2, y2 = N
    
    gradient = (y2 - y1)/(x2 - x1)
    return gradient

def angle_bisector_grad(A, C, P):
    x1, y1 = A
    x3, y3 = C
    x4, y4 = P
    a = gradient(P,A)
    b = gradient(A,C)
    
    grad_angle_bisector1 = (- np.sqrt(a*a + 1) - np.sqrt(b*b + 1))/(a *np.sqrt(b*b +1) +b *np.sqrt(a*a +1))
    grad_angle_bisector2 = (a *np.sqrt(b*b +1) +b *np.sqrt(a*a +1))/(np.sqrt(b*b +1) +np.sqrt(a*a +1))
    return min(grad_angle_bisector1,grad_angle_bisector2)

def vertical_bisector_grad(A,C):
    grad_vertical_bisector = -1/gradient(A,C)
    return grad_vertical_bisector

def M_coordinate_solver(A,mid_point_coor,grad_ang,grad_vert):
    x1, y1 = A
    x_mid, y_mid = mid_point_coor
    k_ang = grad_ang
    k_mid = grad_vert
    
    if(k_ang-k_mid) == 0:
        return nan,nan
    else:
        xm = (y_mid - y1 + k_ang *x1 - k_mid * x_mid)/(k_ang - k_mid)
        ym = k_ang * (xm - x1) +y1
        
        return xm,ym

nodes_index = len(Unqualified_nodes_zeta)
new_nodes_list = []
new_z_list = []
new_o_list = []

for i in range(nodes_index):
    A = (Unqualified_nodes_zetal[i],Unqualified_nodes_omegal[i])
    B = (Unqualified_nodes_zeta[i],Unqualified_nodes_omega[i])
    C = (Unqualified_nodes_zetar[i],Unqualified_nodes_omegar[i])
    P = (Unqualified_nodes_zetall[i],Unqualified_nodes_omegall[i])
    if gradient(P,A) != 0:
        if gradient(A,C) != 0:
            mid_point_coor = mid_point(A,C)
            gradient_angle_bisector = angle_bisector_grad(A, C, P)
            gradient_vertical_bisector = vertical_bisector_grad(A,C)
            solution = M_coordinate_solver(A,mid_point_coor,gradient_angle_bisector,gradient_vertical_bisector)
            new_nodes_list = np.append(new_nodes_list,solution)

j_index = np.linspace(0,int(len(new_nodes_list)),int(len(new_nodes_list)/2+1))
j_index
#len(new_nodes_list)
for j in j_index:
    if j < len(new_nodes_list):
        new_z = new_nodes_list[int(j)]
        new_o = new_nodes_list[int(j+1)]
        new_z_list = np.append(new_z_list,new_z)
        new_o_list = np.append(new_o_list,new_o)
        
#new_z_list
df_new_nodes = pd.DataFrame({'new_nodes_z':new_z_list,'new_nodes_o':new_o_list})
df_new_nodes
df_new_nodes.to_csv("df_new_nodes.csv")

(0.13771325499264, 0.822798307485531)
(0.288488970711566, 0.7118219533516125)
(0.388050097983211, 0.651704493089035)
(0.45999999999999996, 0.5974999999999999)
(0.599044202048696, 0.5438844820521516)
